In [1]:
import requests
import pandas as pd
import hopsworks
import datetime
import matplotlib.pyplot as plt
import json
import re
import os
import sys
from dotenv import load_dotenv
from datetime import datetime, timedelta
import warnings
sys.path.append(os.path.abspath(os.path.join('..', 'functions')))
import util
sys.path.append(os.path.abspath(os.path.join('..', 'model')))
import nn
import torch
from sklearn.preprocessing import StandardScaler


warnings.filterwarnings("ignore")

In [3]:
load_dotenv()
proj = hopsworks.login()
fs = proj.get_feature_store("KTH_ID2223") 

2024-12-26 15:05:53,104 INFO: Closing external client and cleaning up certificates.
Connection closed.
2024-12-26 15:05:53,111 INFO: Initializing external client
2024-12-26 15:05:53,111 INFO: Base URL: https://c.app.hopsworks.ai:443

Multiple projects found. 

	 (1) KTH_ID2223
	 (2) simon003
2024-12-26 15:05:56,702 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1164449


In [4]:
el_prices_fg = fs.get_feature_group(
    name='el_prices',
    version=1,
)
weather_fg = fs.get_feature_group(
    name='weather',
    version=1,
)
power_fg = fs.get_feature_group(
    name='power',
    version=1,
)

In [13]:
#el_prices_fg.show(10)
#weather_fg.show(10)
power_fg.show(10)

Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (0.96s) 


,date,hydro_mw,nuclear_mw,other_mw,wind_mw
0,2023-01-23 00:00:00+00:00,8882.291667,5787.791667,1647.541667,5956.958333
1,2023-12-20 00:00:00+00:00,8903.708333,6893.833333,927.458333,5399.708333
2,2023-05-27 00:00:00+00:00,5346.500000,3149.750000,490.375000,3948.500000
3,2023-03-15 00:00:00+00:00,9685.041667,5773.375000,1438.291667,5425.250000
4,2022-11-01 00:00:00+00:00,8805.291667,5699.208333,897.958333,2831.708333
5,2023-07-27 00:00:00+00:00,5855.166667,5868.750000,421.916667,1487.625000
6,2023-01-26 00:00:00+00:00,8646.083333,5786.083333,1341.875000,6692.166667
7,2024-06-02 00:00:00+00:00,6023.583333,5273.750000,540.583333,2113.666667
8,2023-03-22 00:00:00+00:00,8556.833333,5767.875000,1150.500000,4408.458333
9,2024-09-04 00:00:00+00:00,7250.458333,4541.625000,364.666667,4651.583333


In [5]:
weather_df = weather_fg.read()
power_df = power_fg.read()
el_prices_df = el_prices_fg.read()


Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (0.70s) 
Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (0.61s) 
Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (0.47s) 


In [6]:

merged_df = (
    weather_df
    .merge(power_df, on="date", how="inner")
    .merge(el_prices_df, on="date", how="inner")
)

merged_df['date'] = pd.to_datetime(merged_df['date'])
merged_df = merged_df.sort_values(by="date").reset_index(drop=True)
merged_df['month'] = merged_df['date'].dt.month

dates = merged_df['date']
features = merged_df.drop(columns=['date', 'price'])
target = merged_df['price']

scaler = StandardScaler()
continuous_features = features.drop(columns=['month'])
scaled_continuous_features = scaler.fit_transform(continuous_features)
scaled_features_df = pd.DataFrame(scaled_continuous_features, columns=continuous_features.columns)
scaled_features_df['month'] = features['month'].values

print(scaled_features_df.head())

   temperature_2m_mean  precipitation_sum  wind_speed_10m_max  \
0             0.202644          -0.192797           -0.778611   
1             0.305440          -0.276450           -0.764935   
2             0.105065          -0.527408           -0.485279   
3             0.155420          -0.527408            0.781580   
4             0.076366           1.034108            0.371348   

   wind_direction_10m_dominant  sunshine_duration  hydro_mw  nuclear_mw  \
0                     0.141250          -1.197800  0.587802    0.306440   
1                    -0.103051          -0.240993  0.198458    0.295679   
2                    -0.046024           0.032508  0.169019    0.293499   
3                    -0.555824          -0.097937 -0.361710    0.302126   
4                    -0.419893          -1.197800 -0.378251    0.306301   

   other_mw   wind_mw  month  
0  0.225771 -0.565631     11  
1  0.231565  0.028540     11  
2  0.315731  0.260998     11  
3  0.197290  0.630926     11  
4 -

In [7]:
input_features = [
        "temperature_2m_mean", "precipitation_sum", "wind_speed_10m_max",
        "wind_direction_10m_dominant", "sunshine_duration",
        "Hydro Water Reservoir", "Nuclear", "Other", "Wind Onshore", "month"
]
input_size = len(input_features)
model = nn.EnergyPricePredictorLSTM(input_size=input_size)


In [ ]:
start_date = "2022-11-01"
end_date = "2024-12-20"

inputs, outputs = util.create_time_series(scaled_features_df.values, target)
#Split 70/15/15
num_samples = inputs.shape[0]
train_size = int(0.7 * num_samples)
val_size = int(0.15 * num_samples)

train_inputs = inputs[:train_size]
train_outputs = outputs[:train_size]

val_inputs = inputs[train_size : train_size + val_size]
val_outputs = outputs[train_size : train_size + val_size]

test_inputs = inputs[train_size + val_size :]
test_outputs = outputs[train_size + val_size :]


#Tensors

train_data = torch.tensor(train_inputs, dtype=torch.float32)
train_labels = torch.tensor(train_outputs, dtype=torch.float32)
validation_data = torch.tensor(val_inputs, dtype=torch.float32)
validation_labels = torch.tensor(val_outputs, dtype=torch.float32)
test_data = torch.tensor(test_inputs, dtype=torch.float32)
test_labels = torch.tensor(test_outputs, dtype=torch.float32)




In [9]:
model.train_model(train_inputs, train_outputs, val_inputs, val_outputs)

TypeError: 'int' object is not callable

In [ ]:
predictions, mse = model.test_model(test_data, test_labels)
model.plot_predictions(predictions, test_labels)

download model hopswork